In [1]:
import pandas as pd
from yaml import load
from yaml import CLoader as Loader
from planktonsdm.tune import tune
from planktonsdm.functions import example_data
from sklearn.preprocessing import OneHotEncoder

In [2]:
#If you want to debug the package without repackaging, you can read files locally:

import sys
sys.path.insert(0, '/home/phyto/planktonSDM/planktonsdm/')
from tune import tune
from functions import example_data

In [3]:
# Setting up the model

with open('/home/phyto/planktonSDM/configuration/ensemble.yml', 'r') as f:
    model_config = load(f, Loader=Loader)

d = pd.read_csv("/home/phyto/planktonSDM/data/abundances_environment.csv")
species = 'Umbellosphaera tenuis'
species = 'Emiliania huxleyi'

d.dropna(subset=species, inplace=True)
d.dropna(subset='FID', inplace=True)

features = model_config['predictors']

X_train = d[features]
y = d[species]


try:
    X_train = X_train.drop(columns=['FID'])
    enc = OneHotEncoder()
    regions  = enc.fit_transform(d[['FID']])
    X_train.loc[:, enc.categories_[0]]  = regions.toarray()
    X_train.columns = X_train.columns.astype(str)
    
except:
    print("FID not part of predictors")


m = tune(X_train, y, model_config)

upsampling = True


In [4]:
'''
1-phase Random forest regressor
'''
m.train(model="rf", regressor=True)

training regressor
Fitting 10 folds for each of 72 candidates, totalling 720 fits
exported model to: /home/phyto/CoccoML/ModelOutput/rf/model/Umbellosphaera tenuis_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/rf/scoring/Umbellosphaera tenuis_reg.sav
reg rRMSE: 302%
reg rMAE: 98%
reg R2: 0.43
execution time: 1.1920928955078125e-06 seconds


In [5]:
'''
1-phase XGboost regressor
'''
m.train(model="xgb", regressor=True)

training regressor
Fitting 10 folds for each of 72 candidates, totalling 720 fits
exported model to: /home/phyto/CoccoML/ModelOutput/xgb/model/Umbellosphaera tenuis_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/xgb/scoring/Umbellosphaera tenuis_reg.sav
reg rRMSE: 319%
reg rMAE: 77%
reg R2: 0.33
execution time: 7.152557373046875e-07 seconds


In [6]:
'''
1-phase KNN regressor
'''
m.train(model="knn", regressor=True)

training regressor
Fitting 10 folds for each of 1 candidates, totalling 10 fits
exported model to: /home/phyto/CoccoML/ModelOutput/knn/model/Umbellosphaera tenuis_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/knn/scoring/Umbellosphaera tenuis_reg.sav
reg rRMSE: 318%
reg rMAE: 103%
reg R2: 0.32
execution time: 9.5367431640625e-07 seconds
